**Representation-based Language Models**
---

文章とか文字列を埋め込みなどを紹介してくれる章。

# ![](https://i.gyazo.com/55778936ee06eeba85ea972d8adc801e.png)

データに対して良い埋め込み、表現などが言語モデルの成功に大きく左右するという考え。

# ![](https://i.gyazo.com/0cc1f094057250abe3c0481887b732d4.png)

1はベクトルの成分に複素数が入っている場合の同値関係。

# ![](https://i.gyazo.com/3a2e0b826db7928c4b43f5de3bcf95c3.png)

ヒルベルト空間は距離と角度を考えられるような空間 (例: 二次元ユーグリッド空間)

上の定義を見ると、各要素の和が発散しないようなものをヒルベルト空間と呼ぶ？

任意の内積空間はヒルベルト空間に変換できるらしい。

# ![](https://i.gyazo.com/83f30b4ea13e667ac2384c59607bb015.png)

$\Sigma$の1要素につき、D次元のベクトルで表すよということ。

embeddingの仕方でone-hotの方法あるけど、sparseすぎるため、異なる文字間での類似度が常に0になって、意味的な類似度を捉えることができない。

なので、one-hotみたいに、|文字数|の数だけ行を持てばいいのではなくて、任意のトークン列を受け取って、ベクトルを返すようなものを考える、Attentionなど？

# ![](https://i.gyazo.com/16ca4eb5a1b51c47bb026aafc3558b91.png)

上のものが任意の文字列受け取って、ベクトル空間に移すもの。

$\operatorname{enc}(\overline{\boldsymbol{y}})$と、文字レベルの埋め込み$e(\bar{y})$ の内積はE enc $(\overline{\boldsymbol{y}})$のように表すようにして、これは型がEenc $(\bar{y})$なので、内積ベクトルをスコア(logits)とよぶ。これにsoftmaxなどをかければ、言語モデルのような取り扱いもできる？